# test notebook

In [15]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
from src.kernels import gaussian_kernel,laplacian_kernel
from src.model import KernelRegressor
from src.get_data import get_mnist,inject_label_noise
from sklearn.model_selection import train_test_split
import torch

In [34]:
data,target=get_mnist(n_samples=50000,flatten=True)

X_train,X_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)

In [35]:
print(data.shape)
print(target.shape)
print(target[:4])

torch.Size([50000, 784])
torch.Size([50000])
tensor([8, 1, 2, 9])


In [36]:
print("Training samples:",X_train.shape[0])
print("Test samples:",X_test.shape[0])
print("\nFeature shape:",X_train.shape)
print("Labels shape:",y_train.shape)

Training samples: 40000
Test samples: 10000

Feature shape: torch.Size([40000, 784])
Labels shape: torch.Size([40000])


In [37]:
kernel_reg=KernelRegressor(kernel_func=gaussian_kernel,sigma=5.0)
kernel_reg.invers_fit(X_train,y_train)

print("Kernel matrix shape:",kernel_reg.K.shape)
print("Alpha coefficient shape:", kernel_reg.alpha.shape)

rhks_norm=kernel_reg.compute_rkhs_norm()
print("RKHS norm of the learned function f*:",rhks_norm.item())


Kernel matrix shape: torch.Size([40000, 40000])
Alpha coefficient shape: torch.Size([40000, 10])
RKHS norm of the learned function f*: 170.5939178466797


In [38]:
predictions=kernel_reg.predict(X_train)
predicted_classes=torch.argmax(predictions,dim=1)
accuracy=(predicted_classes==y_train).float().mean().item()

print("Train accuracy:",accuracy)

Train accuracy: 1.0


In [39]:
predictions=kernel_reg.predict(X_test)
predicted_classes=torch.argmax(predictions,dim=1)
accuracy=(predicted_classes==y_test).float().mean().item()

print("Test accuracy:",accuracy)


Test accuracy: 0.9710999727249146


In [ ]:
noise_levels=[0.01,0.1]

for noise in noise_levels:
    noisy_y_train=inject_label_noise(y_train,noise_ratio=noise)
    kernel_reg.invers_fit(X_train,noisy_y_train)
    predictions=kernel_reg.predict(X_test)
    predicted_classes=torch.argmax(predictions,dim=1)
    accuracy=(predicted_classes==y_test).float().mean().item()
    print(f"Noise level: {noise}, Test accuracy: {accuracy:.4f}")
    print(f"RHKS norm:{kernel_reg.compute_rkhs_norm().item()}\n")

Noise level: 0.0, Test accuracy: 0.9705
RHKS norm:171.5307159423828

